# Módulos importados

In [0]:
from pyspark.sql.functions import to_date, min, max, regexp_replace
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName("Spark DF").getOrCreate()

# Criando DataFrames

In [0]:
def build_initial_df(df_type):
    for i, table in enumerate(dbutils.fs.ls(f'/FileStore/tables/csv/{df_type}/')):
        if i != 0:
            df = df.union(spark.read.csv(table.path, header=True, inferSchema=True))
        else:
            df = spark.read.csv(table.path, header=True, inferSchema=True)
    return df

In [0]:
df_cons = build_initial_df(df_type='cons')

In [0]:
df_det = build_initial_df(df_type='det')

In [0]:
det_rows_length = df_det.count()
det_duplicated_length = det_rows_length - df_det.dropDuplicates().count()
cons_rows_length = df_cons.count()
cons_duplicated_length = cons_rows_length - df_cons.dropDuplicates().count()

In [0]:
print(f'Número de observações _DET: {det_rows_length}')
print(f'Número de observações duplicadas: {det_duplicated_length}\n')
print(f'Número de observações _CONS: {cons_rows_length}')
print(f'Número de observações duplicadas: {cons_duplicated_length}')

Número de observações _DET: 39926392
Número de observações duplicadas: 276158

Número de observações _CONS: 6216597
Número de observações duplicadas: 0


A tabela unificada do tipo _DET possui mais de 276 mil registros duplicados. Vamos remover as duplicações para um futuro modelo de Machine Learning não seja prejudicado na fase de aprendizado.

In [0]:
# A tabela unificada do tipo _DET possui mais de 276 mil registros duplicados.
# Vamos remover as duplicações para um futuro modelo de Machine Learning não seja prejudicado na fase de aprendizado.

df_det = df_det.dropDuplicates()

In [0]:
det_rows_length = df_det.count()
det_duplicated_length = det_rows_length - df_det.dropDuplicates().count()
cons_rows_length = df_cons.count()
cons_duplicated_length = cons_rows_length - df_cons.dropDuplicates().count()

In [0]:
print(f'Número de observações _DET: {det_rows_length}')
print(f'Número de observações duplicadas: {det_duplicated_length}\n')
print(f'Número de observações _CONS: {cons_rows_length}')
print(f'Número de observações duplicadas: {cons_duplicated_length}')

Número de observações _DET: 39650234
Número de observações duplicadas: 0

Número de observações _CONS: 6216597
Número de observações duplicadas: 0


# Selecionando Variáveis

Obs: Para fazermos uma primeira análise sobre quais variáveis devem seguir para o modelo preditivo, se faz necessário pesquisar acerca do dicionário de variáveis, disponível no próprio site dos Dados Abertos.<br>
Podemos considerar as seguintes variáveis como não muito informativas para um modelo preditivo:
- CD_TABELA_REFERENCIA
- IND_PACOTE
- IND_TABELA_PROPRIA
- NM_MODALIDADE
- CD_MOTIVO_SAIDA
- CID_1, CID_2, CID_3, CID_4
- LG_VALOR_PREESTABELECIDO
<br>
São todas colunas que dizem respeito aos gestores do sistema de saúde para fins administrativos e que não agregam estatisticamente o suficiente para que sejam consideradas num modelo preditivo.

In [0]:
df_det = df_det.drop('CD_TABELA_REFERENCIA','IND_PACOTE','IND_TABELA_PROPRIA')
df_cons = df_cons.drop('NM_MODALIDADE','CD_MOTIVO_SAIDA','CID_1','CID_2','CID_3','CID_4','LG_VALOR_PREESTABELECIDO')

# Junção (Join) dos DataFrames

In [0]:
# Antes de unirmos os dois DF's, vamos renomear algumas colunas do dataframe _CONS para que não tenha colunas com nome duplicado (mesmo nome no dataframe _DET)
df_cons = df_cons.withColumnRenamed("UF_PRESTADOR", "UF_PRESTADOR_CONS").withColumnRenamed("TEMPO_DE_PERMANENCIA", "TEMPO_DE_PERMANENCIA_CONS").withColumnRenamed("ANO_MES_EVENTO", "ANO_MES_EVENTO_CONS")

In [0]:
full_df = df_det.join(df_cons,['ID_EVENTO_ATENCAO_SAUDE'], 'outer')

In [0]:
# Primeiro, vamos checar se as duas colunas que contêm UF_PRESTADOR, TEMPO_DE_PERMANENCIA e ANO_MES_EVENTO têm dados idênticos (e, portanto, consistentes)
full_df.filter(full_df.UF_PRESTADOR != full_df.UF_PRESTADOR_CONS).count()

Out[189]: 0

In [0]:
full_df.filter(full_df.TEMPO_DE_PERMANENCIA != full_df.TEMPO_DE_PERMANENCIA_CONS).count()

Out[190]: 384341

In [0]:
full_df.filter(full_df.ANO_MES_EVENTO != full_df.ANO_MES_EVENTO_CONS).count()

Out[191]: 1093

In [0]:
full_df = full_df.withColumn('TEMPO_DE_PERMANENCIA', full_df.TEMPO_DE_PERMANENCIA.cast(IntegerType()))

In [0]:
# Existem diferenças consideráveis, principalmente em torno do TEMPO_DE_PERMANENCIA, mas este é até esperado porque uma coluna está com dtype double e outra com dtype int.
# Optaremos por manter a primeira coluna (do Dataframe _DET), pois de acordo com o dicionário de variáveis, esse DataFrame é de preenchimento obrigatório por parte dos funcionários da saúde, enquanto o dataframe _CONS não é obrigatório.
full_df = full_df.drop('UF_PRESTADOR_CONS','TEMPO_DE_PERMANENCIA_CONS','ANO_MES_EVENTO_CONS')

# Análise Exploratória

In [0]:
# Agora faremos uma análise exploratória de cada coluna para compreendermos o que cada uma significa e como lidar com dados faltantes ou inconsistentes.
full_df.columns

Out[195]: ['ID_EVENTO_ATENCAO_SAUDE',
 'UF_PRESTADOR',
 'TEMPO_DE_PERMANENCIA',
 'ANO_MES_EVENTO',
 'CD_PROCEDIMENTO',
 'QT_ITEM_EVENTO_INFORMADO',
 'VL_ITEM_EVENTO_INFORMADO',
 'VL_ITEM_PAGO_FORNECEDOR',
 'ID_PLANO',
 'FAIXA_ETARIA',
 'SEXO',
 'CD_MUNICIPIO_BENEFICIARIO',
 'PORTE',
 'CD_MODALIDADE',
 'CD_MUNICIPIO_PRESTADOR',
 'CD_CARATER_ATENDIMENTO',
 'CD_TIPO_INTERNACAO',
 'CD_REGIME_INTERNACAO',
 'QT_DIARIA_ACOMPANHANTE',
 'QT_DIARIA_UTI',
 'IND_ACIDENTE_DOENCA']

### Coluna ID_EVENTO_ATENCAO_SAUDE

In [0]:
# Essa é nossa variável de identificação dos eventos.
# Não há muito o que checar aqui, mas vamos verificar o tipo da variável e se há valores nulos.
full_df.dtypes[0]

Out[176]: ('ID_EVENTO_ATENCAO_SAUDE', 'bigint')

In [0]:
full_df.filter(full_df.ID_EVENTO_ATENCAO_SAUDE.isNull()).count()

Out[151]: 0

### Coluna UF_PRESTADOR

In [0]:
# Aqui, esperamos dtype 'string'
full_df.dtypes[1]

Out[196]: ('UF_PRESTADOR', 'string')

In [0]:
# Checando se há valores nulos
full_df.filter(full_df.UF_PRESTADOR.isNull()).count()

Out[197]: 15226

In [0]:
# Esperamos 26 estados + 1 Distrito Federal, totalizando 27 valores únicos
full_df.select('UF_PRESTADOR').distinct().count()

### Coluna TEMPO_DE_PERMANENCIA

In [0]:
# De acordo com o dicionário de variáveis, essa coluna traz o tempo de internação em dias.
# Quando não foi possível registrar o tempo, registra-se -1
# Quando a entrada e saída do internamento acontece no mesmo dia, registra-se 1
# Vamos checar o tipo da variável.
full_df.dtypes[2]

In [0]:
# Checando valores nulos
full_df.filter(full_df.TEMPO_DE_PERMANENCIA.isNull()).count()

In [0]:
# Temos mais de 370 mil registros com TEMPO_DE_PERMANENCIA nulos.
# É razoável que se considere essas nulidades como -1, visto que o próprio dicionário de variáveis assim definiu o valor quando não fosse possível fazer essa contabilização.
full_df = full_df.na.fill(-1, subset=['TEMPO_DE_PERMANENCIA'])

### Coluna ANO_MES_EVENTO

In [0]:
# Checando o tipo da variável
full_df.dtypes[3]

In [0]:
# É mais produtivo, rápido e intuitivo trabalhar com variáveis do tipo date do que string, em caso de a coluna representar uma data.
# Para isto, vamos converter a coluna ANO_MES_EVENTO para dtype date
full_df = full_df.withColumn('ANO_MES_EVENTO', to_date(full_df.ANO_MES_EVENTO, 'yyyy-MM'))

In [0]:
# Checando o tipo da variável novamente
full_df.dtypes[3]

In [0]:
# Checando valores nulos
full_df.filter(full_df.ANO_MES_EVENTO.isNull()).count()

### Coluna CD_PROCEDIMENTO

In [0]:
# Esta coluna diz respeito a um identificador dos procedimentos internos ao sistema de saúde.
# De acordo com o dicionário de variáveis, o tipo desta variável deve ser string
# Primeiro, vamos checar o tipo da variável
full_df.dtypes[4]

In [0]:
# Agora, chequemos se há valores nulos
full_df.filter(full_df.CD_PROCEDIMENTO.isNull()).count()

O melhor formato para esta coluna é realmente o de string, visto que, apesar de muito se ver números inteiros, fica claro que ela trata mais de uma variável categórica do que de uma variável discreta.<br>
Apesar de não nos trazer informações mais precisas, esta variável pode ser interessante para um modelo preditivo, visto que por meio do procedimento adotado, podemos estimar o valor financeiro da ocorrência, o tempo de internação etc.<br>
Manteremos a variável por este motivo.

### Coluna QT_ITEM_EVENTO_INFORMADO

In [0]:
# Esta coluna diz respeito à quantidade de procedimentos/itens assistenciais utilizados e deve ser do tipo numérica.
df_det.dtypes[6]

Out[110]: ('QT_ITEM_EVENTO_INFORMADO', 'string')

In [0]:
# Façamos uma análise exploratória para checar porque o PySpark interpretou esta coluna como string
df_det.select('QT_ITEM_EVENTO_INFORMADO').describe().show()

+-------+------------------------+
|summary|QT_ITEM_EVENTO_INFORMADO|
+-------+------------------------+
|  count|                39279430|
|   mean|       55.21354823702275|
| stddev|       5304.467026923824|
|    min|                    ,004|
|    max|                 9999,99|
+-------+------------------------+



Aparentemente, alguns valores são inseridos com vírgula, outros com ponto. Vamos substituir todas as vírgulas por pontos e ver se é possível transformar a coluna em integer.

In [0]:
df_det = df_det.withColumn('QT_ITEM_EVENTO_INFORMADO', regexp_replace('QT_ITEM_EVENTO_INFORMADO',',','.'))

In [0]:
df_det = df_det.withColumn('QT_ITEM_EVENTO_INFORMADO', df_det.QT_ITEM_EVENTO_INFORMADO.cast(IntegerType()))

In [0]:
# Checando novamente o tipo da variável
df_det.dtypes[6]

Out[131]: ('QT_ITEM_EVENTO_INFORMADO', 'int')

In [0]:
# Nova análise descritiva
df_det.select('QT_ITEM_EVENTO_INFORMADO').describe().show()

+-------+------------------------+
|summary|QT_ITEM_EVENTO_INFORMADO|
+-------+------------------------+
|  count|                39279430|
|   mean|       57.95872114233837|
| stddev|       5294.722338418308|
|    min|                       0|
|    max|                32288070|
+-------+------------------------+



In [0]:
# Checando valores nulos
df_det.filter(df_det.QT_ITEM_EVENTO_INFORMADO.isNull()).count()

Out[133]: 370804

In [0]:
# Temos mais de 370 mil 

Esta variável também pode nos fornecer informações importantes acerca do gasto, ou mesmo pode ser utilizado como Target para treinar um modelo preditivo que busca estimar a quantidade de procedimentos/itens necessários para determinado evento. Por este motivo, será mantida.

### Coluna VL_ITEM_EVENTO_INFORMADO

### Coluna VL_ITEM_PAGO_FORNECEDOR